In [ ]:
from pathlib import Path

# User Input

In [ ]:
dir_ = r'E:\Data\Antonio\data\manta\DCR\2017-04-07/'

In [ ]:
[p.name for p in Path(dir_).glob('*.dat')]

In [ ]:
fname = dir_ + '01_dcr.dat'

In [ ]:
description = 'DCR of N5 (donor) and N2 (acceptor) mantas.'

author = 'Maya Segal'
author_affiliation = 'UCLA'
creator = 'Antonino Ingargiola'
creator_affiliation = 'UCLA'

# Imports

In [ ]:
import time
import numpy as np
import tables
from tqdm import tqdm_notebook, tnrange  # progress bar

import phconvert as phc
phc.__version__

In [ ]:
import niconverter as nic
nic.__version__

# Preprocess spot data

In [ ]:
t_start = time.time()

In [ ]:
h5file, meta = nic.ni96ch_process_spots(fname, close=True)
ts_unit = 1 / meta['clock_frequency']
fname, meta

In [ ]:
time.time() - t_start

In [ ]:
h5file = tables.open_file(fname[:-3] + 'hdf5', mode='a')
dt, endianess, meta = nic.detectformat(fname)
ts_unit = 1 / meta['clock_frequency']
t2, aem2 = nic.get_photon_data_arr(h5file, spots=np.arange(48))
meta['acquisition_duration'] = nic.duration(t2, ts_unit)
fname, meta

In [ ]:
time.time() - t_start

In [ ]:
print('Measurement duration: %.1f s' %  meta['acquisition_duration'])
print('Transfer rate: %.2f MB/s' % (meta['num_timestamps'] / meta['acquisition_duration'] * 4 / (1024**2)))

In [ ]:
neg_jumps = []
for tx in tqdm_notebook(t2):
    d = np.diff(tx[:])
    assert (d >= 0).all()
    neg_jumps.append(d[d<0])
    
missed_ov_jumps = [len(j) for j in neg_jumps]
for ch, jumps in enumerate(missed_ov_jumps):
    if jumps > 0:
        print('CH%d: %s' % (ch, jumps))

In [ ]:
time.time() - t_start

# Create Photon-HDF5

In [ ]:
metadata = dict(
    description = description,
    acquisition_duration = meta['acquisition_duration'],

    identity = dict(
        author=author,
        author_affiliation=author_affiliation,
        creator=creator,
        creator_affiliation=creator_affiliation),
    )
metadata

In [ ]:
fname

In [ ]:
h5file.filename

In [ ]:
data = nic.create_ph5data_smFRET_48spots(fname, h5file, ts_unit, metadata)

In [ ]:
[f for f in data.keys() if not f.startswith('photon')]

In [ ]:
%%timeit -n1 -r1
phc.hdf5.save_photon_hdf5(data, h5file=h5file, overwrite=True, close=True)

In [ ]:
time.time() - t_start

In [ ]:
stop 

# More stuff

In [ ]:
if data['_data_file'].isopen:
    data['_data_file'].close()

In [ ]:
h5file = tables.open_file(h5file.filename)

In [ ]:
ts_list, aem = nic.get_photon_data_arr(h5file, range(48))

In [ ]:
neg_jumps = []
for tx in tqdm_notebook(ts_list):
    d = np.diff(tx[:])
    assert (d > 0).all()
    neg_jumps.append(d[d<0])
assert sum(len(j) for j in neg_jumps) == 0

In [ ]:
jump_stats = [(len(j), min(j), max(j), np.mean(j)) if len(j) > 0 else (0, 0, 0, 0) for j in neg_jumps]

for ch, jumps in enumerate(jump_stats):
    if jumps[0] > 0:
        print('CH%02d: %4d [%4d %4d] <%d>' % (ch, *jumps))